# KNN

## Motivation

The principle behind nearest neighbor methods is to find a predefined number of training samples closest in distance to the new point, and predict the label from these. The number of samples can be a user-defined constant (k-nearest neighbor learning), or vary based on the local density of points (radius-based neighbor learning). The distance can, in general, be any metric measure: standard Euclidean distance is the most common choice. Neighbors-based methods are known as non-generalizing machine learning methods, since they simply “remember” all of its training data

~[scikit-learn](http://scikit-learn.org/stable/modules/neighbors.html)

It's a beautiful day in this neighborhood,
A beautiful day for a neighbor.
Would you be mine?
Could you be mine?

~ Mr. Rogers

**Readings**: 
* openCV: http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_ml/py_knn/py_knn_understanding/py_knn_understanding.html
* dataquest: https://www.dataquest.io/blog/k-nearest-neighbors/  
* k-d tree: https://ashokharnal.wordpress.com/2015/01/20/a-working-example-of-k-d-tree-formation-and-k-nearest-neighbor-algorithms/
* euclidean: http://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/


## Data

In [ ]:
import pandas
import numpy
#from scipy.stats import mode
from sklearn import neighbors
from sklearn.neighbors import DistanceMetric 
from pprint import pprint

MY_TITANIC_TRAIN = '/media/removable/data/train_titanic.csv'
MY_TITANIC_TEST = '/media/removable/data/test_titanic.csv'
titanic_dataframe = pandas.read_csv(MY_TITANIC_TRAIN, header=0)
print('length: {0} '.format(len(titanic_dataframe)))
titanic_dataframe.head(5)

* Remove Columns

In [ ]:
titanic_dataframe.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
print('dropped')

In [ ]:
titanic_dataframe.describe()

* Which are the factors?

In [ ]:
titanic_dataframe.info()

Pre-Processing

In [ ]:
titanic_dataframe = titanic_dataframe.dropna()

titanic_dataframe['Embarked'].unique()
titanic_dataframe['Port'] = titanic_dataframe['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)


titanic_dataframe['Sex'].unique()
titanic_dataframe['Gender'] = titanic_dataframe['Sex'].map({'female': 0, 'male': 1}).astype(int)
titanic_dataframe = titanic_dataframe.drop(['Sex', 'Embarked'], axis=1)


In [ ]:

cols = titanic_dataframe.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
titanic_dataframe = titanic_dataframe[cols]

train_data = titanic_dataframe[cols[2: ]]
train_target = titanic_dataframe[cols[1]]

pprint('column_list: {0}'.format(cols))
print(train_data.head(2))

In [ ]:
df_test = pandas.read_csv(MY_TITANIC_TEST)

df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

df_test = df_test.dropna()

df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test['Port'] = df_test['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)

df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

test_data = df_test.values



### Normalize & Fill

*  Kaggle needs all rows: alternates to dropping na rows.
*  scale features

## Evaluation

### [Confusion Matrix*](https://en.wikipedia.org/wiki/Confusion_matrix)

<table class="wikitable" align="center" style="text-align:center; border:none; background:transparent;">
<tr>
<td colspan="2" style="border:none;"></td>
<td colspan="2" style="background:#eeeebb;"><b>Predicted condition</b></td>
</tr>
<tr>
<td style="border:none;"></td>
<td style="background:#dddddd;"><a href="https://en.wikipedia.org/wiki/Statistical_population" title="Statistical population">Total population</a></td>
<td style="background:#ffffcc;">Predicted Condition positive</td>
<td style="background:#ddddaa;">Predicted Condition negative</td>
<td style="background:#eeeecc;"><a href="https://en.wikipedia.org/wiki/Prevalence" title="Prevalence">Prevalence</a> <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ&#160;Condition positive</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Total population</span></span></span></span></td>
</tr>
<tr>
<td rowspan="2" style="background:#bbeeee;"><b>True<br />
condition</b></td>
<td style="background:#ccffff;">condition<br />
positive</td>
<td style="background:#ccffcc;"><span style="color:#006600;"><b><a href="https://en.wikipedia.org/wiki/True_positive" title="True positive" class="mw-redirect">True positive</a></b></span></td>
<td style="background:#eedddd;"><span style="color:#cc0000;"><b><a href="https://en.wikipedia.org/w/index.php?title=False_Negative&amp;action=edit&amp;redlink=1" class="new" title="False Negative (page does not exist)">False Negative</a></b></span><br />
(<a href="https://en.wikipedia.org/wiki/Type_II_error" title="Type II error" class="mw-redirect">Type II error</a>)</td>
<td style="background:#eeffcc;"><a href="https://en.wikipedia.org/wiki/True_positive_rate" title="True positive rate" class="mw-redirect">True positive rate</a> (TPR), <a href="https://en.wikipedia.org/wiki/Sensitivity_(tests)" title="Sensitivity (tests)" class="mw-redirect">Sensitivity</a>, <a href="https://en.wikipedia.org/wiki/Recall_(information_retrieval)" title="Recall (information retrieval)" class="mw-redirect">Recall</a> <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ True positive</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Condition&#160;positive</span></span></span></span></td>
<td style="background:#ffeecc;"><a href="https://en.wikipedia.org/wiki/False_negative_rate" title="False negative rate" class="mw-redirect">False negative rate</a> (FNR), Miss&#160;rate <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ False negative</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Condition&#160;positive</span></span></span></span></td>
</tr>
<tr>
<td style="background:#aadddd;">condition<br />
negative</td>
<td style="background:#ffdddd;"><span style="color:#cc0000;"><b><a href="https://en.wikipedia.org/wiki/False_Positive" title="False Positive" class="mw-redirect">False Positive</a></b></span><br />
(<a href="https://en.wikipedia.org/wiki/Type_I_error" title="Type I error" class="mw-redirect">Type I error</a>)</td>
<td style="background:#bbeebb;"><span style="color:#006600;"><b><a href="https://en.wikipedia.org/wiki/True_negative" title="True negative" class="mw-redirect">True negative</a></b></span></td>
<td style="background:#eeddbb;"><a href="https://en.wikipedia.org/wiki/False_positive_rate" title="False positive rate">False positive rate</a> (FPR), <a href="https://en.wikipedia.org/wiki/Information_retrieval" title="Information retrieval"><span class="nowrap">Fall-out</span></a> <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ False positive</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Condition&#160;negative</span></span></span></span></td>
<td style="background:#ddeebb;"><a href="https://en.wikipedia.org/wiki/True_negative_rate" title="True negative rate" class="mw-redirect">True negative rate</a> (TNR), <a href="https://en.wikipedia.org/wiki/Specificity_(tests)" title="Specificity (tests)" class="mw-redirect">Specificity</a> (SPC) <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ True negative</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Condition&#160;negative</span></span></span></span></td>
</tr>
<tr>
<td style="border:none;"></td>
<td rowspan="2" style="background:#cceecc;border-top:solid grey;border-right:solid grey;"><a href="https://en.wikipedia.org/wiki/Accuracy_and_precision" title="Accuracy and precision">Accuracy</a> (ACC) <span style="font-size:118%;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ&#160;True positive + Σ True negative</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Total population</span></span></span></span></td>
<td style="background:#ccffee;border-top:solid grey;"><a href="https://en.wikipedia.org/wiki/Positive_predictive_value" title="Positive predictive value" class="mw-redirect">Positive predictive value</a> (PPV), <a href="https://en.wikipedia.org/wiki/Precision_(information_retrieval)" title="Precision (information retrieval)" class="mw-redirect">Precision</a> <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ True positive</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Test&#160;outcome&#160;positive</span></span></span></span></td>
<td style="background:#eeddee;border-bottom:solid grey;"><a href="https://en.wikipedia.org/wiki/False_omission_rate" title="False omission rate" class="mw-redirect">False omission rate</a> (FOR) <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ False negative</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Test&#160;outcome&#160;negative</span></span></span></span></td>
<td style="background:#eeeeee;"><a href="https://en.wikipedia.org/wiki/Positive_likelihood_ratio" title="Positive likelihood ratio" class="mw-redirect">Positive likelihood ratio</a> <span class="nowrap">(LR+)</span> <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">TPR</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">FPR</span></span></span></span></td>
<td rowspan="2" style="background:#dddddd;"><a href="https://en.wikipedia.org/wiki/Diagnostic_odds_ratio" title="Diagnostic odds ratio">Diagnostic odds ratio</a> (DOR) <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">LR+</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">LR−</span></span></span></span></td>
</tr>
<tr>
<td style="border:none;"></td>
<td style="background:#cceeff;border-top:solid grey;"><a href="https://en.wikipedia.org/wiki/False_discovery_rate" title="False discovery rate">False discovery rate</a> (FDR) <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ False positive</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Test&#160;outcome&#160;positive</span></span></span></span></td>
<td style="background:#aaddcc;border-bottom:solid grey;"><a href="https://en.wikipedia.org/wiki/Negative_predictive_value" title="Negative predictive value" class="mw-redirect">Negative predictive value</a> (NPV) <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">Σ True negative</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">Σ&#160;Test&#160;outcome&#160;negative</span></span></span></span></td>
<td style="background:#cccccc;"><a href="https://en.wikipedia.org/wiki/Negative_likelihood_ratio" title="Negative likelihood ratio" class="mw-redirect">Negative likelihood ratio</a> <span class="nowrap">(LR−)</span> <span style="font-size:118%;white-space:nowrap;">= <span class="texhtml"><span class="sfrac nowrap" style="display:inline-block; vertical-align:-0.5em; font-size:85%; text-align:center;"><span style="display:block; line-height:1em; margin:0 0.1em;">FNR</span><span style="display:block; line-height:1em; margin:0 0.1em; border-top:1px solid;">TNR</span></span></span></span></td>
</tr>
</table>

[* Clarification of the Confusion Matrix](http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology)

## Representation

### Distance

#### Factors/Categorical Data
```python
hamming_distance(row, train_row):
    distance = list(
        1 
        if type(input_row[i]) is str & not training_row[i]==input_row[i]) else 0
        for i in range(len(input_row)
    )

    return sum(distance)
```

In [ ]:
factors = [ 'Pclass', 'Port', 'Gender']
hamming = DistanceMetric.get_metric('hamming')
spamming = []


#### Euclidian

$\sqrt{\Sigma{(q_i-p_i)^2}}$
```python
euclidian_distance(row, train_row):
    distance = list((row[i] - train_row[i])^2
                     for i in range(len(input_row)) if isnumeric(row[i]) & isnumeric(train_row[i])
               )
    return sqrt(sum(distance))
```

In [ ]:
numericals = ['Age', 'SibSp', 'Parch', 'Fare']
euclidian = DistanceMetric.get_metric('euclidian')
euclidish =[]

### KNN
```python
knn(row, train, k):
    distance = []
    for train_row in train:
        hamming = hamming_distance(row, train_row[factors])
        euclidian  = euclidian (row, train_row[numericals])
        distance.append(hamming + euclidian)
    distance.sort(key=operator.itemgetter(0))
    out = distance[ :k]
    return (row[id], 1 if sum(out) > k//2 else 0)
```

## Optimization

* [KNN Algorithms](http://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbor-algorithms)

* Tuning K

## SciKit Learn

In [ ]:
model = neighbors.KNeighborsClassifier()
model.fit(train_data.values, train_target.values)

result = numpy.c_[test_data[:,0].astype(int), output.astype(int)]


df_result = pandas.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])
df_result.to_csv('titanic.csv', index=False) 